In [ ]:
import os

os.getcwd()



'c:\\Users\\dsteinec\\Documents\\Jupyter_Notebooks\\nl_to_data'

### SQL Database

In [6]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = "data/chinook.db"
conn = sqlite3.connect(db_path)

# --- Tables ---
tables = pd.read_sql_query("""
SELECT name FROM sqlite_master
WHERE type='table' AND name NOT LIKE 'sqlite_%'
ORDER BY name;
""", conn)

# --- Columns ---
columns = pd.read_sql_query("""
SELECT
    m.name AS table_name,
    p.name AS column_name,
    p.type AS data_type,
    p.pk AS primary_key
FROM sqlite_master m
JOIN pragma_table_info(m.name) p
WHERE m.type = 'table'
  AND m.name NOT LIKE 'sqlite_%'
ORDER BY m.name, p.cid;
""", conn)

tables, columns

(              name
 0           albums
 1          artists
 2        customers
 3        employees
 4           genres
 5    invoice_items
 6         invoices
 7      media_types
 8   playlist_track
 9        playlists
 10          tracks,
    table_name   column_name      data_type  primary_key
 0      albums       AlbumId        INTEGER            1
 1      albums         Title  NVARCHAR(160)            0
 2      albums      ArtistId        INTEGER            0
 3     artists      ArtistId        INTEGER            1
 4     artists          Name  NVARCHAR(120)            0
 ..        ...           ...            ...          ...
 59     tracks       GenreId        INTEGER            0
 60     tracks      Composer  NVARCHAR(220)            0
 61     tracks  Milliseconds        INTEGER            0
 62     tracks         Bytes        INTEGER            0
 63     tracks     UnitPrice  NUMERIC(10,2)            0
 
 [64 rows x 4 columns])

### Generate Schema Documents

In [2]:
from generate_schema_documents import make_schema_documents

# Generate schema documents
documents = make_schema_documents(conn)
print(documents[0].text)

KeyError: 'samples'

### Embed Documents in a Vector Database

In [19]:
from embed_documents_into_vector_db import upsert_schema_docs_to_lancedb, SchemaDoc

vector_db = upsert_schema_docs_to_lancedb(
                db_dir="chinook_schema_docs",
                documents=documents)
print(vector_db.count_rows())

86


In [20]:
results = vector_db.search("How is the customer table linked?").limit(5).to_pydantic(SchemaDoc)

In [22]:
results

[SchemaDoc(id='table:customers', doc_type='table', table='customers', column='', ref_table='', ref_column='', text='Table: customers\nPrimary key: CustomerId\nColumns:\n - CustomerId (INTEGER) [PK NOT NULL]\n - FirstName (NVARCHAR(40)) [NOT NULL]\n - LastName (NVARCHAR(20)) [NOT NULL]\n - Company (NVARCHAR(80))\n - Address (NVARCHAR(70))\n - City (NVARCHAR(40))\n - State (NVARCHAR(40))\n - Country (NVARCHAR(40))\n - PostalCode (NVARCHAR(10))\n - Phone (NVARCHAR(24))\n - Fax (NVARCHAR(24))\n - Email (NVARCHAR(60)) [NOT NULL]\n - SupportRepId (INTEGER)\nRelationships (foreign keys):\n - customers.SupportRepId → employees.EmployeeId', vector=FixedSizeList(dim=384)),
 SchemaDoc(id='table:invoices', doc_type='table', table='invoices', column='', ref_table='', ref_column='', text='Table: invoices\nPrimary key: InvoiceId\nColumns:\n - InvoiceId (INTEGER) [PK NOT NULL]\n - CustomerId (INTEGER) [NOT NULL]\n - InvoiceDate (DATETIME) [NOT NULL]\n - BillingAddress (NVARCHAR(70))\n - BillingCity (N

In [ ]:
from ctransformers import AutoModelForCausalLM
import sqlite3

from generate_schema_documents import make_schema_documents
from embed_documents_into_vector_db import upsert_schema_docs_to_lancedb, \
    get_relevant_documents 
from create_sql_query import generate_sql_cpu, docs_to_context_string, \
    extract_table_descriptions


if __name__ == "__main__":
    query = "What is the first song by the first artist?"

    # --- Connect to the database
    db_path = "data/chinook.db"
    conn = sqlite3.connect(db_path)

    # --- Take SQL tables and columns, then generate "documents" for each
    # NOTE: Later add LLM-generated descriptions for each table/column
    documents = make_schema_documents(conn)
    print(f'---Example Document---\n{documents[0].text}\n')

    # --- Embed documents in a vector database NOTE: max(<thresh, 5)
    vector_db = upsert_schema_docs_to_lancedb(
                    db_dir="chinook_schema_docs",
                    documents=documents)
    print(f'{vector_db.count_rows()} documents in vector database')

    # --- Validate document relevance
    # NOTE: Later, use another model to validate that documents match the query
    
    # --- Retrieve relevant columns from vector database
    sql_context_df = get_relevant_documents(vector_db, query)
    print(f'{len(sql_context_df)} relevant documents found for "{query}"')
    table_descriptions = extract_table_descriptions(sql_context_df, documents)
    sql_context = docs_to_context_string(sql_context_df)
    sql_context += f'\n{table_descriptions}'
    
    # --- Generate SQL query
    model = AutoModelForCausalLM.from_pretrained(
        "MaziyarPanahi/sqlcoder-7b-2-GGUF",
        model_file="sqlcoder-7b-2.Q4_K_M.gguf", 
        model_type="mistral", 
        gpu_layers=0,  
        context_length=4096,
    )
    sql_query = generate_sql_cpu(
        question="Who are the top 3 artists?", 
        retrieved_docs=sql_context, model=model)
    print(sql_query)
    

---Example Document---
Table: albums
Primary key: AlbumId
Columns:
 - AlbumId (INTEGER) [PRIMARY KEY NOT NULL] (ex: 1, 4, 2)
 - Title (NVARCHAR(160)) [NOT NULL] (ex: For Those About To Rock We Salute You, Balls to the Wall, Restless and Wild)
 - ArtistId (INTEGER) [NOT NULL] (ex: 1, 2, 3)
Foreign key(s):
 - albums.ArtistId → artists.ArtistId

86 documents in vector database
3 relevant documents found for "What is the first song by the first artist?"


Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


Generating SQL...
 SELECT a.ArtistName FROM artists a ORDER BY a.ArtistName ASC NULLS LAST LIMIT 3


In [11]:
print(sql_context)

[DOCUMENT_START]
Table: albums
Primary key: AlbumId
Columns:
 - AlbumId (INTEGER) [PRIMARY KEY NOT NULL] (ex: 1, 4, 2)
 - Title (NVARCHAR(160)) [NOT NULL] (ex: For Those About To Rock We Salute You, Balls to the Wall, Restless and Wild)
 - ArtistId (INTEGER) [NOT NULL] (ex: 1, 2, 3)
Foreign key(s):
 - albums.ArtistId → artists.ArtistId[DOCUMENT_END]

[DOCUMENT_START]
Column: albums.ArtistId
Data type: INTEGER
Nullable: no
Primary key: no
Default: None
Sample values: 1, 2, 3
[DOCUMENT_END]

[DOCUMENT_START]
Column: albums.Title
Data type: NVARCHAR(160)
Nullable: no
Primary key: no
Default: None
Sample values: For Those About To Rock We Salute You, Balls to the Wall, Restless and Wild
[DOCUMENT_END]





Dimensions of XX^T: (500, 500)
Dimensions of X^T X: (1000, 1000)

Top 5 Eigenvalues from XX^T (Sample Space):
[2867.41231015 2856.83225367 2794.27358665 2783.5793239  2762.78522812]

Top 5 Eigenvalues from X^T X (Feature Space):
[2867.41231015 2856.83225367 2794.27358665 2783.5793239  2762.78522812]

Are the top 500 eigenvalues identical? True

Proportion of Variance Explained by PC1 (XX^T): 0.005741
Proportion of Variance Explained by PC1 (X^T X): 0.005741


In [9]:
import pandas as pd

pd.read_sql_query(sql_query, conn)

DatabaseError: Execution failed on sql ' SELECT a.ArtistName FROM artists a ORDER BY a.ArtistName ASC NULLS LAST LIMIT 3': no such column: a.ArtistName

In [ ]:
### Upload to GitHub
### then add the table context to the top columns
### then re-rank the top ones

In [ ]:
# NOTE: Try GPU version later

In [13]:
import pandas as pd

In [14]:
print('hi')

hi
